# Getting People Data
For use with the clickhandler

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
res = db.run_query("g.V().hasLabel('pop').has('username','BillmanLocal2').valueMap()")
pops = pd.DataFrame([db.clean_node(n) for n in res])
pops

,name,objid,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,Se Bemon,8692749194716,0.298,0.034,1.080,0.932,0.5,6103221482049,1.0060,0.5200,0.256,True,BillmanLocal2,pop,8692749194716
1,Ralplahi Deni,3697596910545,0.266,0.245,0.766,0.852,0.5,9230448847637,0.8090,0.5270,0.336,True,BillmanLocal2,pop,3697596910545
2,Hahuamarikay Hinram,9451256864767,0.388,0.204,0.616,0.878,0.5,0232294329079,0.7470,0.4755,0.290,True,BillmanLocal2,pop,9451256864767
3,Ralplahi Kreis,8677609612767,0.199,0.366,0.726,0.822,0.5,9230448847637,0.7740,0.5700,0.126,True,BillmanLocal2,pop,8677609612767
4,Bia Pi,6720568407752,0.111,0.222,0.503,0.594,0.5,9093605982634,0.5485,0.3852,0.210,True,BillmanLocal2,pop,6720568407752
5,Hahuamarikay Puwan,5957118078765,0.222,0.128,0.649,0.900,0.5,0232294329079,0.7745,0.4512,0.180,True,BillmanLocal2,pop,5957118078765


In [3]:
popID = '4253777177342'
res = db.run_query(f"g.V().hasLabel('pop').has('objid','{popID}').outE().inV().dedup().values('name','label','name')")
res

[]

In [39]:
query = """
g.V().has('objid','7615388501660').as('location')
	.in('inhabits').as('population')
	.local(
		union(
			out('isInFaction').as('faction'),
			out('isOfSpecies').as('species')
			)
			.fold()).as('faction','species')
			.path()
			.by(unfold().valueMap().fold())
            """
# query = """
# g.V().has('objid','7615388501660').as('location')
# 	.in('inhabits').as('population')
# 	.out('isInFaction').as('faction')

# 			.path()
# 			.by(unfold().valueMap().fold())
#             """

res = db.run_query(query)


In [40]:
pd.DataFrame(res)

,labels,objects
0,"[[location], [population], [faction, species]]","[[{'objid': ['7615388501660'], 'name': ['Plara..."
1,"[[location], [population], [faction, species]]","[[{'objid': ['7615388501660'], 'name': ['Plara..."
2,"[[location], [population], [faction, species]]","[[{'objid': ['7615388501660'], 'name': ['Plara..."
3,"[[location], [population], [faction, species]]","[[{'objid': ['7615388501660'], 'name': ['Plara..."
4,"[[location], [population], [faction, species]]","[[{'objid': ['7615388501660'], 'name': ['Plara..."
5,"[[location], [population], [faction, species]]","[[{'objid': ['7615388501660'], 'name': ['Plara..."


In [42]:

from functools import reduce
import operator

def clean_node(x):
    for k in list(x.keys()):
        if len(x[k]) == 1:
            x[k] = x[k][0]
    if 'objid' in x.keys():
        x["id"] = x["objid"]
    return x

fab = {}
for n,r in enumerate(res): 
    fab[n] = {}
    labels = reduce(operator.concat, r['labels'])
    objects = reduce(operator.concat, r['objects'])
    for i,l in enumerate(labels):
        fab[n][l]=clean_node(objects[i])

fab
        




{0: {'location': {'objid': '7615388501660',
   'name': 'Plara',
   'class': 'terrestrial',
   'radius': 0.6865,
   'mass': 0.1512,
   'orbitsDistance': 0,
   'orbitsId': '3442412221426',
   'orbitsName': '',
   'isSupportsLife': 'True',
   'isPopulated': 'True',
   'type': 'terrestrial',
   'isHomeworld': 'True',
   'username': 'BillmanLocal2',
   'objtype': 'planet',
   'id': '7615388501660'},
  'population': {'name': 'Se Bemon',
   'objid': '8692749194716',
   'conformity': 0.298,
   'literacy': 0.034,
   'aggression': 1.08,
   'constitution': 0.932,
   'health': 0.5,
   'isInFaction': '6103221482049',
   'industry': 1.006,
   'wealth': 0.52,
   'factionLoyalty': 0.256,
   'isIdle': 'True',
   'username': 'BillmanLocal2',
   'objtype': 'pop',
   'id': '8692749194716'},
  'faction': {'name': 'Se',
   'objid': '6103221482049',
   'username': 'BillmanLocal2',
   'objtype': 'faction',
   'id': '6103221482049'},
  'species': {'name': 'Wan',
   'objid': '0199620302205',
   'consumes': 'org